In [1]:
import pathlib
import os
import time


DATA_ROOT = pathlib.Path(os.curdir).resolve().parent.joinpath("data")
ARROW_PATH = DATA_ROOT.joinpath("input", "combined.arrow")
OUT_DIR = DATA_ROOT.joinpath("output")
assert ARROW_PATH.exists(), OUT_DIR.exists()

In [2]:
import polars
import vaex

In [19]:
raw_df = polars.scan_ipc(ARROW_PATH)
raw_df.schema

{'file': Utf8,
 'AREA_PIXELS_COUNT': Float64,
 'AREA_UM2': Float64,
 'BBOX_HEIGHT': Float64,
 'BBOX_WIDTH': Float64,
 'BBOX_XMIN': Float64,
 'BBOX_YMIN': Float64,
 'CENTROID_X': Float64,
 'CENTROID_Y': Float64,
 'ENERGY': Float64,
 'ENTROPY': Float64,
 'HYPERFLATNESS': Float64,
 'HYPERSLEWNESS': Float64,
 'INTEGRATED_INTENSITY': Float64,
 'INTERQUARTILE_RANGE': Float64,
 'KURTOSIS': Float64,
 'MASS_DISPLACEMENT': Float64,
 'MAX': Float64,
 'MEAN': Float64,
 'MEAN_ABSOLUTE_DEVIATION': Float64,
 'MEDIAN': Float64,
 'MIN': Float64,
 'MODE': Float64,
 'P01': Float64,
 'P10': Float64,
 'P25': Float64,
 'P75': Float64,
 'P90': Float64,
 'P99': Float64,
 'RANGE': Float64,
 'ROBUST_MEAN_ABSOLUTE_DEVIATION': Float64,
 'ROOT_MEAN_SQUARED': Float64,
 'SKEWNESS': Float64,
 'STANDARD_DEVIATION': Float64,
 'STANDARD_ERROR': Float64,
 'UNIFORMITY': Float64,
 'intensity_image': Utf8,
 'label': Int64,
 'mask_image': Utf8,
 'virus_negative': Float64,
 'virus_neutral': Float64,
 'otsu': Float64,
 'fpr': 

In [15]:
raw_df.head().collect()

file,AREA_PIXELS_COUNT,AREA_UM2,BBOX_HEIGHT,BBOX_WIDTH,BBOX_XMIN,BBOX_YMIN,CENTROID_X,CENTROID_Y,ENERGY,ENTROPY,HYPERFLATNESS,HYPERSLEWNESS,INTEGRATED_INTENSITY,INTERQUARTILE_RANGE,KURTOSIS,MASS_DISPLACEMENT,MAX,MEAN,MEAN_ABSOLUTE_DEVIATION,MEDIAN,MIN,MODE,P01,P10,P25,P75,P90,P99,RANGE,ROBUST_MEAN_ABSOLUTE_DEVIATION,ROOT_MEAN_SQUARED,SKEWNESS,STANDARD_DEVIATION,STANDARD_ERROR,UNIFORMITY,intensity_image,label,mask_image,virus_negative,virus_neutral,otsu,fpr,nsigma
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,str,f64,f64,f64,f64,f64
"""p001_x01_y01_w...",54.0,37.75122,8.0,8.0,673.0,12.0,676.148148,15.5,3.969186e6,2.487264,13.603715,-4.128704,14618.0,22.349333,2.841984,1.362863,296.0,270.703704,12.481481,270.5,228.0,284.0,240.9472,253.092,261.546667,283.896,288.588,295.9728,68.0,11.657143,271.115187,-0.513902,15.07169,2.050997,122.0,"""p001_x01_y01_w...",1,"""p001_x01_y01_w...",0.0,0.0,0.0,0.0,0.0
"""p001_x01_y01_w...",22.0,15.380127,5.0,5.0,872.0,18.0,873.909091,19.909091,2.298055e6,2.283327,4.254393,0.607762,7079.0,45.58,1.917369,1.389226,381.0,321.772727,26.702479,327.0,275.0,275.0,275.3816,283.162,300.44,346.02,366.478,0.0,106.0,25.44,323.198152,0.057446,31.034438,6.616564,26.0,"""p001_x01_y01_w...",2,"""p001_x01_y01_w...",0.0,0.0,0.0,0.0,0.0
"""p001_x01_y01_w...",45.0,31.45935,8.0,7.0,603.0,20.0,606.333333,23.6,4.080029e6,2.466373,7.231118,-1.797631,13521.0,29.52625,2.383321,1.47565,335.0,300.466667,16.34963,303.0,256.0,290.0,256.7505,272.59,287.4025,316.92875,325.52,0.0,79.0,13.166667,301.110131,-0.358209,19.897008,2.966071,71.0,"""p001_x01_y01_w...",3,"""p001_x01_y01_w...",0.0,0.0,0.0,0.0,0.0
"""p001_x01_y01_w...",9.0,6.29187,4.0,3.0,1077.0,21.0,1078.222222,22.333333,828874.0,1.891061,2.197393,0.098674,2712.0,78.225,1.664156,1.498755,352.0,301.333333,31.703704,291.0,247.0,291.0,247.6195,263.17,271.9375,350.1625,351.58,0.0,105.0,26.25,303.475058,0.095859,38.173944,12.724648,11.0,"""p001_x01_y01_w...",4,"""p001_x01_y01_w...",0.0,0.0,0.0,0.0,0.0
"""p001_x01_y01_w...",22.0,15.380127,5.0,5.0,950.0,22.0,952.272727,24.045455,1.59572e6,2.242573,4.965928,0.585515,5920.0,17.16,2.190879,1.44651,289.0,269.090909,9.008264,268.0,250.0,266.0,250.2808,254.953,263.65,280.81,288.6685,0.0,39.0,9.36,269.319007,0.144787,11.342751,2.418283,28.0,"""p001_x01_y01_w...",5,"""p001_x01_y01_w...",0.0,0.0,0.0,0.0,0.0


In [23]:
row_0 = raw_df.select(
    polars.col("file"),
    polars.col("intensity_image"),
    polars.col("mask_image"),
).collect().row(0)
row_0

('p001_x01_y01_wx1_wy1_c2.csv',
 'p001_x01_y01_wx1_wy1_c2.ome.tif',
 'p001_x01_y01_wx1_wy1_c1.ome.tif')

# Columns

1. "file": pattern: "p{ppp}_x{xx}_y{yy}_wx{i}_wy{j}_c{c}.csv"
    - p: plate number
    - x: x co-ordinate of well in plate. Range is [1, 24]
    - y: y co-ordinate of well in plate. Range is [1, 16]
    - i: x co-ordinate of FOV in well. Range is [1, 3]
    - j: y co-ordinate of FOV in well. Range is [1, 3]
    - c: channel. 1 is nuclear/DAPI mask channel, 2 is reporter/covid-expression channel
3. "AREA_PIXELS_COUNT": number of pixels in the mask object
4. "AREA_UM2": given the physical area of each pixel, this is the area in units `micro-meter-squared`
5. "BBOX_HEIGHT": from the bounding box of the mask
6. "BBOX_WIDTH": from the bounding box of the mask
7. "BBOX_XMIN": from the bounding box of the mask
8. "BBOX_YMIN": from the bounding box of the mask
9. "CENTROID_X": from the bounding box of the mask?
10. "CENTROID_Y": from the bounding box of the mask?
11. "ENERGY":
12. "ENTROPY":
13. "HYPERFLATNESS":
14. "HYPERSLEWNESS":
15. "INTEGRATED_INTENSITY":
16. "INTERQUARTILE_RANGE":
17. "KURTOSIS":
18. "MASS_DISPLACEMENT":
19. "MAX":
20. "MEAN":
21. "MEAN_ABSOLUTE_DEVIATION":
22. "MEDIAN":
23. "MIN":
24. "MODE":
25. "P01":
26. "P10":
27. "P25":
28. "P75":
29. "P90":
30. "P99":
31. "RANGE":
32. "ROBUST_MEAN_ABSOLUTE_DEVIATION":
33. "ROOT_MEAN_SQUARED":
34. "SKEWNESS":
35. "STANDARD_DEVIATION":
36. "STANDARD_ERROR":
37. "UNIFORMITY":
38. "intensity_image": name of reporter intensity image: "p{ppp}_x{xx}_y{yy}_wx{i}_wy{j}_c2.ome.tif"
39. "label": nuclear label. Ranges from 1 to n where n is the number of nuclei in the FOV
40. "mask_image": name of nuclear intensity image, e.g. "p{ppp}_x{xx}_y{yy}_wx{i}_wy{j}_c1.ome.tif"
41. "virus_negative":
42. "virus_neutral":
43. "otsu":
44. "fpr":
45. 'nsigma': Float64,

In [5]:
selectors = [
    polars.col("AREA_PIXELS_COUNT").quantile(v / 100).alias(f"APC-quantile-{v}")
    for v in range(10, 100, 10)
]

start = time.perf_counter()
quantiles: polars.DataFrame = (
    raw_df.lazy()
        .select(
            polars.col("AREA_PIXELS_COUNT").min().alias(f"APC-quantile-min"),
            polars.col("AREA_PIXELS_COUNT").quantile(1 / 100).alias(f"APC-quantile-01"),
            *selectors,
            polars.col("AREA_PIXELS_COUNT").quantile(99 / 100).alias(f"APC-quantile-99"),
            polars.col("AREA_PIXELS_COUNT").max().alias(f"APC-quantile-max"),
        )
        .collect()
)
time_taken = time.perf_counter() - start
print(f"{time_taken = :.6f} seconds")

time_taken = 49.663235 seconds


In [6]:
quantiles.head()

APC-quantile-min,APC-quantile-01,APC-quantile-10,APC-quantile-20,APC-quantile-30,APC-quantile-40,APC-quantile-50,APC-quantile-60,APC-quantile-70,APC-quantile-80,APC-quantile-90,APC-quantile-99,APC-quantile-max
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,11.0,26.0,32.0,37.0,42.0,48.0,55.0,64.0,77.0,101.0,232.0,1.1664e6


In [11]:
apc_df = raw_df.lazy().select(polars.col("AREA_PIXELS_COUNT").cast(polars.UInt32)).collect()
apc_df.head()

AREA_PIXELS_COUNT
u32
54
22
45
9
22


In [12]:
start = time.perf_counter()
quantiles: polars.DataFrame = (
    apc_df.lazy()
        .select(
            polars.col("AREA_PIXELS_COUNT").min().alias(f"APC-quantile-min"),
            polars.col("AREA_PIXELS_COUNT").quantile(1 / 100).alias(f"APC-quantile-01"),
            *selectors,
            polars.col("AREA_PIXELS_COUNT").quantile(99 / 100).alias(f"APC-quantile-99"),
            polars.col("AREA_PIXELS_COUNT").max().alias(f"APC-quantile-max"),
        )
        .collect()
)
time_taken = time.perf_counter() - start
print(f"{time_taken = :.6f} seconds")

time_taken = 14.153679 seconds


In [13]:
quantiles.head()

APC-quantile-min,APC-quantile-01,APC-quantile-10,APC-quantile-20,APC-quantile-30,APC-quantile-40,APC-quantile-50,APC-quantile-60,APC-quantile-70,APC-quantile-80,APC-quantile-90,APC-quantile-99,APC-quantile-max
u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32
1,11.0,26.0,32.0,37.0,42.0,48.0,55.0,64.0,77.0,101.0,232.0,1166400
